In [7]:
import pandas as pd
import requests as r
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.express as px
from utils.settings import DETERMINISTIC_MODELS, DETERMINISTIC_VARS, ENSEMBLE_VARS, ENSEMBLE_MODELS, DEFAULT_TEMPLATE
import plotly.io as pio
import plotly.express as px

In [150]:
def get_ensemble_data(latitude=53.55,
                      longitude=9.99,
                      variables=",".join(ENSEMBLE_VARS),
                      timezone='auto',
                      model=ENSEMBLE_MODELS[0]['value'],
                      from_now=True):
    """
    Get the ensemble data
    """
    if model == 'icon_seamless':
        forecast_days = 7
    else:
        forecast_days = 14
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": variables,
        "timezone": timezone,
        "models": model,
        "forecast_days": forecast_days
    }

    resp = r.get("https://ensemble-api.open-meteo.com/v1/ensemble",
                 params=payload)
    resp.raise_for_status()
    data = pd.DataFrame.from_dict(resp.json()['hourly'])
    data['time'] = pd.to_datetime(
        data['time']).dt.tz_localize(resp.json()['timezone'], ambiguous='NaT', nonexistent='NaT')

    data = data.dropna(subset=data.columns[data.columns != 'time'],
                       how='all')
    # Optionally subset data to start only from previous hour
    if from_now:
        data = data[data.time >= pd.to_datetime(
            'now', utc=True).tz_convert(resp.json()['timezone']).floor('H')]

    return data

def get_forecast_data(latitude=53.55,
                      longitude=9.99,
                      variables=",".join(DETERMINISTIC_VARS),
                      timezone='auto',
                      model=DETERMINISTIC_MODELS[0]['value'],
                      forecast_days=7,
                      from_now=True,
                      past_days=None):
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": variables,
        "timezone": timezone,
        "models": model,
        "forecast_days": forecast_days
    }

    if past_days:
        payload['past_days'] = past_days

    resp = r.get("https://api.open-meteo.com/v1/forecast",
                 params=payload)
    resp.raise_for_status()
    data = pd.DataFrame.from_dict(resp.json()['hourly'])
    data['time'] = pd.to_datetime(
        data['time']).dt.tz_localize(resp.json()['timezone'], ambiguous='NaT')

    data = data.dropna(subset=data.columns[data.columns != 'time'],
                       how='all')
    # Optionally subset data to start only from previous hour
    if from_now:
        data = data[data.time >= pd.to_datetime(
            'now', utc=True).tz_convert(resp.json()['timezone']).floor('H')]

    return data

def get_forecast_daily_data(latitude=53.55,
                            longitude=9.99,
                            variables="precipitation_sum",
                            timezone='auto',
                            model=DETERMINISTIC_MODELS[0]['value'],
                            forecast_days=7,
                            past_days=None):
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": variables,
        "timezone": timezone,
        "models": model,
        "forecast_days": forecast_days
    }

    if past_days:
        payload['past_days'] = past_days

    resp = r.get("https://api.open-meteo.com/v1/forecast",
                 params=payload)
    resp.raise_for_status()
    data = pd.DataFrame.from_dict(resp.json()['daily'])
    data['time'] = pd.to_datetime(
        data['time']).dt.tz_localize(resp.json()['timezone'], ambiguous='NaT', nonexistent='NaT')

    return data

In [122]:
data = get_forecast_data(model='ecmwf_ifs04,gfs_seamless,jma_seamless,icon_seamless,gem_seamless,meteofrance_seamless',
                         variables=",".join(DETERMINISTIC_VARS),
                         forecast_days=14)

In [149]:
sun = get_forecast_data(model='ecmwf_ifs04',
                         variables="sunrise,sunset",
                         forecast_days=14)

HTTPError: 400 Client Error: Bad Request for url: https://api.open-meteo.com/v1/forecast?latitude=53.55&longitude=9.99&hourly=sunrise%2Csunset&timezone=auto&models=ecmwf_ifs04&forecast_days=14

In [121]:
DETERMINISTIC_VARS = ["temperature_2m", "relativehumidity_2m",
                      "precipitation", "rain", "showers", "snowfall", "snow_depth",
                      "cloudcover", "windspeed_10m", "winddirection_10m", "windgusts_10m",
                      "is_day"]

In [142]:
# Function to compute daytime intervals
def compute_daytime_intervals(df):
    is_day_columns = data.columns[data.columns.str.contains('is_day')][0]
    intervals = []
    start_time = None

    for index, row in df.iterrows():
        if row[is_day_columns] == 1:
            if start_time is None:
                start_time = row['time']
        else:
            if start_time is not None:
                intervals.append((start_time, row['time']))
                start_time = None

    if start_time is not None:
        intervals.append((start_time, df['time'].max()))

    return intervals

# Compute daytime intervals
daytime_intervals = compute_daytime_intervals(data)

In [156]:
sun = get_forecast_daily_data(variables='sunrise,sunset',
                              forecast_days=14)

,time,sunrise,sunset
0,2023-11-14 00:00:00+01:00,2023-11-14T07:43,2023-11-14T16:25
1,2023-11-15 00:00:00+01:00,2023-11-15T07:45,2023-11-15T16:23
2,2023-11-16 00:00:00+01:00,2023-11-16T07:47,2023-11-16T16:22
3,2023-11-17 00:00:00+01:00,2023-11-17T07:49,2023-11-17T16:20
4,2023-11-18 00:00:00+01:00,2023-11-18T07:50,2023-11-18T16:19
5,2023-11-19 00:00:00+01:00,2023-11-19T07:52,2023-11-19T16:17
6,2023-11-20 00:00:00+01:00,2023-11-20T07:54,2023-11-20T16:16
7,2023-11-21 00:00:00+01:00,2023-11-21T07:56,2023-11-21T16:15
8,2023-11-22 00:00:00+01:00,2023-11-22T07:58,2023-11-22T16:14
9,2023-11-23 00:00:00+01:00,2023-11-23T07:59,2023-11-23T16:12


In [165]:
def make_lineplot_timeseries(df, var, mode='lines+markers', showlegend=False):
    traces = []
    # Define cyclical colors to be used
    colors = pio.templates[DEFAULT_TEMPLATE]['layout']['colorway'] * 5
    for i, col in enumerate(df.columns[df.columns.str.contains(var)]):
        traces.append(
            go.Scatter(
                x=df.loc[:, 'time'],
                y=df.loc[:, col],
                mode=mode,
                name=col.replace(var+"_",''),
                marker=dict(size=5, color=colors[i]),
                line=dict(width=2, color=colors[i]),
                showlegend=showlegend),
        )

    return traces

def make_barplot_timeseries(df, var):
    traces = []
    # Define cyclical colors to be used
    colors = pio.templates[DEFAULT_TEMPLATE]['layout']['colorway'] * 5
    for i, col in enumerate(df.columns[df.columns.str.contains(var)]):
        traces.append(
            go.Bar(
                x=df['time'],
                y=df[col],
                name=col.replace(var+"_",''),
                opacity=0.6,
                marker=dict(color='rgb(26, 118, 255)'),
                showlegend=False),
        )
        traces.append(
            go.Scatter(
                x=df.loc[df[col] >= 0.1, 'time'],
                y=df.loc[df[col] >= 0.1, col],
                mode='markers',
                name=col.replace(var+"_",''),
                marker=dict(size=3, color=colors[i]),
                showlegend=False),
        )
    

    return traces

def make_subplot_figure(data, title=None, sun=None):
    traces_temp = make_lineplot_timeseries(data, 'temperature_2m', showlegend=True)
    traces_precipitation = make_barplot_timeseries(data, 'precipitation')
    traces_wind = make_lineplot_timeseries(data, 'windspeed_10m')
    traces_cloud = make_lineplot_timeseries(data, 'cloudcover', mode='markers')

    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.5, 0.3, 0.3, 0.3])

    for trace_temp in traces_temp:
        fig.add_trace(trace_temp, row=1, col=1)
    for trace_precipitation in traces_precipitation:
        fig.add_trace(trace_precipitation, row=2, col=1)
    for trace_wind in traces_wind:
        fig.add_trace(trace_wind, row=3, col=1)
    for trace_cloud in traces_cloud:
        fig.add_trace(trace_cloud, row=4, col=1)

    fig.update_layout(
        xaxis=dict(showgrid=True,
                   range=[data['time'].min(),
                          data['time'].max()]),
        yaxis=dict(showgrid=True,),
        height=1000,
        margin={"r": 5, "t": 40, "l": 0.1, "b": 0.1},
        barmode='overlay',
        legend=dict(orientation='h',y=-0.04),
    )

    if sun is not None:
        for i, s in sun.iterrows():
            fig.add_vrect(
                x0=s['sunrise'],
                x1=s['sunset'],
                fillcolor="rgba(255, 255, 0, 0.3)",
                layer="below",
                line=dict(width=0),
                row=1, col=1
            )

    fig.update_yaxes(title_text="2m Temp [°C]", row=1, col=1)
    fig.update_yaxes(title_text="Prec. [mm]", row=2, col=1)
    fig.update_yaxes(title_text="Wind. [kph]", row=3, col=1)
    fig.update_yaxes(title_text="Cloud cover [%]", row=4, col=1)
    fig.update_yaxes(showgrid=True, gridwidth=4)
    fig.update_xaxes(minor=dict(ticks="inside", showgrid=True,
                     gridwidth=1),
                     tickformat='%a %d %b\n%H:%M', showgrid=True, gridwidth=4)
    if title is not None:
        fig.update_layout(title_text=title)

    return fig

In [166]:
make_subplot_figure(data, sun=sun)